# Trigger Ingest via Cumulus

The following code demonstrates how to create a token for calling the Cumulus API /rules endpoint.

The Cumulus Rules API is used to trigger workflows defined as part of an existing Cumulus deployment.

This assumes the collection has already been published to Cumulus and CMR.

In [1]:
import datetime
import json
import requests

#### 1. Define some parameters and functions for calling the Cumulus API

In [9]:
# Root Cumulus API and the other routes we will need to use
cumulus_rest_api = 'https://vpnm371s8e.execute-api.us-west-2.amazonaws.com/dev'
authorize_url = f"{cumulus_rest_api}/token"
ingest_url = f"{cumulus_rest_api}/rules"

# Collection and provider-specific parameters
collection_name = 'dit_testing'
collection_version = '001'
provider = 'maap-dit-workspace'

# Login credentials for calling the Cumulus API
# Note: this requires these users exist in the Cumulus Users table
earthdata_username = 'aimeeb'
earthdata_password = 'ADD ME'

In [3]:
def get_redirect_url():
    response = requests.get(authorize_url, allow_redirects=False)
    return response.headers['location']

In [4]:
def generate_user_token():
    session = requests.Session()
    session.auth = (earthdata_username, earthdata_password)
    tokens_url = get_redirect_url()
    auth_resp = session.get(
        tokens_url,
        headers={
            "Accept": "application/json",
        },
        timeout=10,
    )
    return auth_resp

#### 2. Generate the token

In [5]:
# Generate the token and headers
token_response = generate_user_token()
token = token_response.json()['message']['token']

headers = {
  'Content-Type': 'application/json',
  'Authorization': f"Bearer {token}"
}

#### 3. Generate the rules API POST body

In [6]:
d = int(datetime.datetime.now().timestamp())
body = {
  'workflow': 'DiscoverAndQueueGranulesWorkflow',
  'collection': {
    'name': collection_name,
    'version': collection_version
  },
  'provider': provider,
  'name': f"ade___{collection_name}_{d}",
  'rule': {
    'type': 'onetime',
    'value': ''
  },
  'meta': {},
  'state': 'ENABLED'
}
body

{'workflow': 'DiscoverAndQueueGranulesWorkflow',
 'collection': {'name': 'dit_testing', 'version': '001'},
 'provider': 'maap-dit-workspace',
 'name': 'ade___dit_testing_1664211835',
 'rule': {'type': 'onetime', 'value': ''},
 'meta': {},
 'state': 'ENABLED'}

#### 4. Call and inspect the response

In [8]:
response = requests.post(ingest_url, data = json.dumps(body), headers = headers)
print(f"Received response: {response.status_code}: {response.content}")

Received response: 200: b'{"message":"Record saved","record":{"createdAt":1664211871945,"updatedAt":1664211871945,"workflow":"DiscoverAndQueueGranulesWorkflow","collection":{"name":"dit_testing","version":"001"},"provider":"maap-dit-workspace","name":"ade___dit_testing_1664211835","rule":{"type":"onetime","value":""},"meta":{},"state":"ENABLED"}}'
